In [1]:
#Reread in all of the data and group by star
import gzip 
with gzip.open('AllVar.phot.gz', 'rt') as f:
    temp = f.read()
temp2 = temp.split("\n")
del temp2[-1]
dataa = [[float(y) for y in x.split(',')] for x in temp2]
import itertools
import numpy as np 
groups = itertools.groupby(dataa, lambda x: x[0])
grouped_data = [list(g) for k,g in groups]

#Get the maximum size of a time-series
lengths = [len(list(zip(*a))[2]) for a in grouped_data]
l_pad = max(lengths)

In [2]:
#Allocate space for both the padded time series and the length of the unpadded features (will be used later for RNN)
feat_pad = np.empty((len(grouped_data),l_pad,2))
feat_len = np.empty(len(grouped_data))
i = 0

In [3]:
#Iterate through all stars and pad with zeros
for a in grouped_data:
    tmp = np.array(sorted(zip(list(zip(*a))[1],list(zip(*a))[2]))) 
    tmp = np.column_stack((np.diff(tmp[:,0], prepend=tmp[0,0]),tmp[:,1]))
    feat_pad[i,0:tmp.shape[0],:] = tmp
    feat_len[i] = tmp.shape[0]
    i += 1

In [20]:
#SDAE from https://github.com/rajarsheem/libsdae-autoencoder-tensorflow/blob/master/deepautoencoder/stacked_autoencoder.py

In [4]:
import tensorflow as tf

In [6]:
truth = np.load("true_labels.npy")
labels = np.zeros((truth.size, truth.max()))
labels[np.arange(truth.size),truth-1] = 1
# Training Parameters
learning_rate = 0.0025
epochs = 10
batch_size = 128
display_step = 1

# Network Parameters
num_input = 2 
timesteps = l_pad 
num_hidden = 59 # hidden layer num of features
num_classes = labels.shape[1] 

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [1]:
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define 2 lstm cells with tensorflow
    nh = [num_hidden, num_hidden]
    cells = [tf.nn.rnn_cell.LSTMCell(num_hidden) for n in nh]
    lstm_cell = tf.nn.rnn_cell.MultiRNNCell(cells)
    #lstm_cell = tf.nn.rnn_cell.LSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = tf.nn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

NameError: name 'tf' is not defined

In [13]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [ ]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    ind = np.arange(feat_pad.shape[0])
    for step in range(0, epochs):
        np.random.shuffle(ind)
        for i in range(0, feat_pad.shape[0], batch_size):
            batch_x = feat_pad[ind[i:i+batch_size],:,:]
            batch_y = labels[ind[i:i+batch_size],:]
        # Run optimization op (backprop)
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            #loss, acc = sess.run([loss_op, accuracy], feed_dict={X: feat_pad,
            #                                                     Y: labels})
            #print("Step " + str(step) + ", Minibatch Loss= " + \
            #      "{:.4f}".format(loss) + ", Training Accuracy= " + \
            #      "{:.3f}".format(acc)).
            print("Epoch" + str(step))

    loss, acc = sess.run([loss_op, accuracy], feed_dict={X: feat_pad,Y: labels})
    print("Accuracy =" + "{:.3f}".format(acc))
    print("Optimization Finished!")